# Portfolio Project: Berlin Restaurants

For this Data analysis project, I used the [YELP-FUSION](https://www.yelp.com/developers/documentation/v3) API to extract the restaurants based in the urban area of [Berlin](https://www.yelp.de/berlin) with all the information that comes with it. This project showcases the ETL process that is followed for each data-based project. 


## Enter Pipeline

The solution to this problem is a data pipeline. Like a physical pipeline system that drives the flow of any material
from the source to the destination, a data pipeline comes to transfer the flow of data from any source to the desired
destination. But not only that, it can make all the necessary transformations -while transferring the data- so the data reach their destination in the desired format and ready to be analyzed.

<p align="center">
    <img alt="Pipeline" src="https://mermaid.ink/img/eyJjb2RlIjoiXG5ncmFwaCBURFxuc3ViZ3JhcGggRGVzdGluYXRpb25cbkUoRGF0YSBXYXJlaG91c2UpXG5lbmRcbkEoQ2xvdWQgQnVja2V0KSAtLVBpcGVsaW5lLS0-IEVcbkIoTW9iaWxlIEFwcCkgLS1QaXBlbGluZS0tPiBFXG5DKEFQSSkgLS1QaXBlbGluZS0tPiBFXG5EKExvY2FsIERhdGFiYW5rKSAtLVBpcGVsaW5lLS0-IEVcblxuIiwibWVybWFpZCI6eyJ0aGVtZSI6ImRlZmF1bHQifSwidXBkYXRlRWRpdG9yIjpmYWxzZX0">
</p>

For this project, the source is just an API and the destination would be a Postgres SQL database. So the pipeline for this project would look like this

<p align="center">
    <img alt="Pipeline" src="https://mermaid.ink/img/eyJjb2RlIjoiZ3JhcGggTFJcbnN1YmdyYXBoIERlc3RpbmF0aW9uXG5DKFNRTCBEQilcbmVuZFxuc3ViZ3JhcGggRVRMXG5Ce1RyYW5zb3JtYXRpb25zfVxuZW5kXG5zdWJncmFwaCBTb3VyY2VcbkEoQVBJKVxuZW5kXG5BLS1QaXBlbGluZS0tPiBCXG5CLS1QaXBlbGluZS0tPkNcbiIsIm1lcm1haWQiOnsidGhlbWUiOiJkZWZhdWx0In0sInVwZGF0ZUVkaXRvciI6ZmFsc2V9">
</p>


Each API lets us make some GET/POST requests and sends back to us a payload with the appropriate information depending on what we send it. The logic behind each API is the same but each one uses a unique style of commands that need to be sent with each request to receive the correct response.

Yelp's API has the following format

```python
 response = requests.get(api_url, params=params, headers=headers)
```

In the params dictionary, we have to define what do we want to look for, for this case it would be the following

```python
params = {"term": "restaurants",
          "location": "berlin"}
```

Passing to the key terms: `term`:`restaurants` and `location`:`Berlin`, we ask the API to look for the term restaurants located in Berlin. 

The request is almost ready, it requires more information. Although it's a public API, someone must be registered to use the API. After registration you will receive a KEY, this key should be passed to the request with the following text into the headers like this

```python
headers = {"Authorization": "Bearer {}".format(api_key)}
```









<div class="list-group" id="list-tab" role="tablist">
    <h3 class="list-group-item list-group-item-action active" data-toggle="list" role="tab" aria-controls="home">
        Notebook Content!</h3>
    <a class="list-group-item list-group-item-action" data-toggle="list" href="#libraries" role="tab"
        aria-controls="profile">Import Libraries<span class="badge badge-primary badge-pill">1</span></a>
    <a class="list-group-item list-group-item-action" data-toggle="list" href="#connect-API" role="tab"
        aria-controls="profile">Connect to the API <span class="badge badge-primary badge-pill">2</span></a>
    <a class="list-group-item list-group-item-action" data-toggle="list" href="#connect-db" role="tab"
        aria-controls="messages">Connect to the SQL Database<span
            class="badge badge-primary badge-pill">3</span></a>
    <a class="list-group-item list-group-item-action" data-toggle="list" href="#data-cleaning" role="tab"
        aria-controls="messages">Data Cleaning<span
            class="badge badge-primary badge-pill">4</span></a>
    <a class="list-group-item list-group-item-action" data-toggle="list" href="#sales-analysis" role="tab"
        aria-controls="messages">Data Analysis<span
            class="badge badge-primary badge-pill">5</span></a>
    <a class="list-group-item list-group-item-action" data-toggle="list" href="#final-coclusion" role="tab"
        aria-controls="messages">Conclusion<span
            class="badge badge-primary badge-pill">5</span></a>
</div>

### Import libraries and set constants  <a name='libraries'  href='#list-tab'><span>🔼</span></a>

In [1]:
# Libraries
import os
import psycopg2
import requests
import numpy as np
import pandas as pd
from IPython.display import display
from pandas import json_normalize
from sqlalchemy import create_engine

%load_ext nb_black

<IPython.core.display.Javascript object>

### Connect to the API   <a name='connect-API'  href='#list-tab'><span>🔼</span></a>

As was said before, to make a request we need some credentials it is wise not to share in public projects. There a lot of ways to hide this from the public. I will demonstrate two of them.

**Extract credentials to the environment**

This way demands the creation of a bat file with all the commands needed to set a variable to the local environment, after doing that it is easy to retrieve this information in our code without fear of revealing it to the public.

*.bat file
>```cmd
>set API_KEY = 'my-private-key'
>```

Run the file
>```cmd
>my_bat.bat
>```

**Make a file and import it in your project**

This way is kind of more pythonic. It takes only the creation of a new file where each variable will have the values of some value needed for our project and has to stay private.

environment_consts.py file
>```python
>API_KEY = 'api key'
>```

Project file
>```python
>from environment_consts import API_KEY
>```


In [2]:
# Set the values
import constants

os.environ["API_KEY"] = constants.API_KEY
os.environ["HOST"] = constants.HOST
os.environ["DATABASE_NAME"] = constants.DATABASE_NAME
os.environ["DB_USER"] = constants.DB_USER
os.environ["DB_PASSWORD"] = constants.DB_PASSWORD

<IPython.core.display.Javascript object>

In [3]:
api_url = "https://api.yelp.com/v3/businesses/search"
params = {"term": "restaurants", "location": "berlin"}
api_key = os.environ.get("API_KEY", "")
headers = {"Authorization": "Bearer {}".format(api_key)}


def extract_data(offset=[0]):
    """
    Makes GET requests to the API and recieves restaurants back, each call returns around 20 entries, 
    thats why we use the offset parameter to take different number of responses each time.
    
    :returns: data_list, a list which each element is a json format entry with information about restaurants
    """

    data_list = []

    print("Extracting data from the API...")
    for each in offset:
        params["offset"] = each
        response = requests.get(api_url, params=params, headers=headers)
        data = response.json()
        data_list.append(data)

    return data_list

<IPython.core.display.Javascript object>

In [13]:
# Call the function
restaurants = extract_data(offset=list(range(0, 240, 20)))


Extracting data from the API...


<IPython.core.display.Javascript object>

In [14]:
print(f"Lenght of the list: {len(restaurants)}")
print()
print("First entry:")
(restaurants[0]["businesses"][0])

Lenght of the list: 12

First entry:


{'id': '8uBE9ZgfZOM2z_Rllw478Q',
 'alias': 'stadtklause-berlin',
 'name': 'Stadtklause',
 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/vqFLzToYyFfAoYKlBRq0jA/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/stadtklause-berlin?adjust_creative=N9q5J26XY3aaHLZrjJ7g7g&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=N9q5J26XY3aaHLZrjJ7g7g',
 'review_count': 172,
 'categories': [{'alias': 'german', 'title': 'German'}],
 'rating': 4.5,
 'coordinates': {'latitude': 52.504779, 'longitude': 13.380618},
 'transactions': [],
 'price': '€',
 'location': {'address1': 'Bernburger Str. 35',
  'address2': '',
  'address3': None,
  'city': 'Berlin',
  'zip_code': '10963',
  'country': 'DE',
  'state': 'BE',
  'display_address': ['Bernburger Str. 35', '10963 Berlin', 'Germany']},
 'phone': '+493051056381',
 'display_phone': '+49 30 51056381',
 'distance': 2205.244050414826}

<IPython.core.display.Javascript object>

We see that each entry has a dictionary format. Now it's time to extract from this dictionary the information we need. We can manage this with the `json_normalize` method of the panda's library. The method returns a pandas data frame where each column is a key of the dictionary and each value is the value of that key.

The problem with using this method is that our dictionary is deep nested. This means that there are other dictionaries nested into the dictionary. To capture correctly this morphology wee will use some attributes of the json_nomralize method.

In [15]:
# Raw transform of the dictionary to dataframe
display(json_normalize(restaurants[0]["businesses"][0]).columns)

display(json_normalize(restaurants[0]["businesses"][0]))

Index(['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count',
       'categories', 'rating', 'transactions', 'price', 'phone',
       'display_phone', 'distance', 'coordinates.latitude',
       'coordinates.longitude', 'location.address1', 'location.address2',
       'location.address3', 'location.city', 'location.zip_code',
       'location.country', 'location.state', 'location.display_address'],
      dtype='object')

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address
0,8uBE9ZgfZOM2z_Rllw478Q,stadtklause-berlin,Stadtklause,https://s3-media3.fl.yelpcdn.com/bphoto/vqFLzT...,False,https://www.yelp.com/biz/stadtklause-berlin?ad...,172,"[{'alias': 'german', 'title': 'German'}]",4.5,[],...,52.504779,13.380618,Bernburger Str. 35,,None,Berlin,10963,DE,BE,"[Bernburger Str. 35, 10963 Berlin, Germany]"


<IPython.core.display.Javascript object>

In [16]:
record_path = "categories"
meta = [
    "id",
    "name",
    "price",
    "review_count",
    "rating",
    "phone",
    "distance",
    ["coordinates", "latitude"],
    ["coordinates", "longitude"],
    ["location", "city"],
    ["location", "zip_code"],
    ["location", "address1"],
]
meta_prefix = "restaurant_"
record_prefix = "category_"

<IPython.core.display.Javascript object>

In [17]:
# clean transform of the dictionary to dataframe
json_normalize(
    restaurants[0]["businesses"][0],
    sep="_",
    record_path=record_path,
    meta=meta,
    errors="ignore",
    meta_prefix=meta_prefix,
    record_prefix=record_prefix,
)

,category_alias,category_title,restaurant_id,restaurant_name,restaurant_price,restaurant_review_count,restaurant_rating,restaurant_phone,restaurant_distance,restaurant_coordinates_latitude,restaurant_coordinates_longitude,restaurant_location_city,restaurant_location_zip_code,restaurant_location_address1
0,german,German,8uBE9ZgfZOM2z_Rllw478Q,Stadtklause,€,172,4.5,+493051056381,2205.24405,52.504779,13.380618,Berlin,10963,Bernburger Str. 35


<IPython.core.display.Javascript object>

In [18]:
def transform_data(data):
    """
    Transforming the data for each reastaurant to dataframe where each key is a column.
    
    :returns - A panda's dataframe
    """
    df_list = []
    for each in data:
        df_list.append(
            json_normalize(
                each["businesses"],
                sep="_",
                record_path=record_path,
                meta=meta,
                errors="ignore",
                meta_prefix=meta_prefix,
                record_prefix=record_prefix,
            )
        )
    final_df = pd.concat(df_list, ignore_index=True)
    print("Final data frame created...")
    print(final_df.info())
    return final_df

<IPython.core.display.Javascript object>

In [19]:
restaurants_df = transform_data(restaurants)

Final data frame created...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 14 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   category_alias                    400 non-null    object
 1   category_title                    400 non-null    object
 2   restaurant_id                     400 non-null    object
 3   restaurant_name                   400 non-null    object
 4   restaurant_price                  314 non-null    object
 5   restaurant_review_count           400 non-null    object
 6   restaurant_rating                 400 non-null    object
 7   restaurant_phone                  400 non-null    object
 8   restaurant_distance               400 non-null    object
 9   restaurant_coordinates_latitude   400 non-null    object
 10  restaurant_coordinates_longitude  400 non-null    object
 11  restaurant_location_city          400 non-null    object

<IPython.core.display.Javascript object>

In [20]:
display(restaurants_df.sample(10))

,category_alias,category_title,restaurant_id,restaurant_name,restaurant_price,restaurant_review_count,restaurant_rating,restaurant_phone,restaurant_distance,restaurant_coordinates_latitude,restaurant_coordinates_longitude,restaurant_location_city,restaurant_location_zip_code,restaurant_location_address1
160,spanish,Spanish,6UUxSr7fDTT4tH8Wk0AeeQ,Picoteo,€€,27,4.5,+493060407400,3894.408941,52.4977,13.4131,Berlin,10999,Erkelenzdamm 47
193,breakfast_brunch,Breakfast & Brunch,sEsT-fXXmvnmxKOmrjBxsA,Käfer,€€€,111,4.0,+49302262990,643.361155,52.518619,13.37631,Berlin,11011,Platz der Republik 1
286,thai,Thai,Yiv2p7StX3_b2rchYMLKbQ,Hangmee,€€,30,4.0,+493082717447,5664.92192,52.513,13.45755,Berlin,10245,Boxhagener Str. 108
175,coffee,Coffee & Tea,0fikR43JcEVdeRLNY4MBdQ,Mr. Vertigo,€,48,4.5,+491636010110,827.787597,52.529873,13.384146,Berlin,10115,Chausseestr. 17
109,sandwiches,Sandwiches,BB9cYpDwCX7FU_UwnQ4B-w,Simply...,€€,7,4.5,+493094402603,3921.910909,52.5343,13.4315,Berlin,10407,Pasteurstr. 2
57,scottish,Scottish,FqjkhdhJIpxAVFGfytu73A,Hirsch & Hase,NaN,5,5.0,+4917631322073,2281.968223,52.542081,13.393723,Berlin,13355,Brunnenstr. 73
7,sandwiches,Sandwiches,UzXq4QuasWnooTzHQlsMJg,Muse,€€,60,4.5,+493040056289,3352.020979,52.53299,13.42322,Berlin,10405,Immanuelkirchstr. 31
10,chocolate,Chocolatiers & Shops,MqiobbxpvGTciQk75G9u2w,Rausch Schokoladenhaus,€€,309,4.5,+4930757880,1719.063263,52.51199,13.39135,Berlin,10117,Charlottenstr. 60
291,greek,Greek,EIfQJCBkggq75CkFr3LaTQ,Taverna Amphipolis,€€,35,4.5,+49303951556,2356.677119,52.52854,13.34169,Berlin,10551,Wilhelmshavener Str. 6
370,kebab,Kebab,boBcF__ECw8h5TZuEneI_w,Mustafa Demir's Gemüse Kebap,NaN,18,4.0,,1760.619441,52.522799,13.401745,Berlin,10178,Neue Promenade 7


<IPython.core.display.Javascript object>

In [21]:
# Check if there are duplicated entries
assert sum(restaurants_df.duplicated()) == 0

<IPython.core.display.Javascript object>

It worked, we have an organized data frame with no duplicated values that holds all the information needed for our purposes. We managed to declutter the noise and keep only what is important for our analysis.

✔️Load the data

✔️Transform the data

We have checked two very important steps, now next step is to load the data into a SQL database



### Connect to the database   <a name='connect-db'  href='#list-tab'><span>🔼</span></a>

For this project, I choose to work with PostgreSQL and I used the [Psycopg](https://www.psycopg.org/docs/index.html)
python library to connect to and execute queries to my database. We can execute SQL queries either using the [pgAdmin](https://www.pgadmin.org/) tool or using python.

If we want to run out DB from python we need to export into the environment all the parameters used for the
a connection such as: *the database name*, *password*, *host*, *, etc*, but this step is already completed.

To execute a query from python we need to make a connection with the DB, after that, we create a `cursor`, an object that deals with the execution of each query. In the end, we need to close the connection. For our luck pandas supports the transfer of a data frame to SQL database automatically but only using the SQLAlchemy library. For only this step we have to use this specific library

In [22]:
host = os.environ.get("HOST", "")
dbname = os.environ.get("DATABASE_NAME", "")
user = os.environ.get("DB_USER", "")
password = os.environ.get("DB_PASSWORD", "")

<IPython.core.display.Javascript object>

In [23]:
# Build the template for the connection
connection_template = (
    f"host={host} dbname={dbname} user={user} password={password} sslmode=disable"
)

<IPython.core.display.Javascript object>

In [24]:
# Connect
connection = psycopg2.connect(connection_template)
connection.set_session(autocommit=True)
cursor = connection.cursor()
cursor.execute("SELECT current_database()")
db_name = cursor.fetchone()
print("You are connected with {} database".format(db_name[0]))

You are connected with yelp database


<IPython.core.display.Javascript object>

The message proves that we are connected to the database with the name yelp. This means that now we can run queries from python

In [25]:
def load():
    # Save DF to a CSV file locally
    restaurants_df.to_csv('./restaurants_df.csv', header=True, index_label='id', index=True, encoding='utf-8')

    # Make a connection with the SQLalchemy package
    connect = "postgresql+psycopg2://%s:%s@%s:5432/%s" % (
        os.environ.get("DB_USER"),
        os.environ.get("DB_PASSWORD"),
        os.environ.get("HOST"),
        os.environ.get('DATABASE_NAME')
    )
    # Create engine
    engine = create_engine(connect)

    restaurants_df.to_sql(name="raw_table",con=engine,if_exists="replace",index=True)


<IPython.core.display.Javascript object>

In [26]:
# Create a local file with the data, and also push all data to sql database
load()

<IPython.core.display.Javascript object>

In [28]:
# Test if everything is fine
q = """
    SELECT * FROM raw_table
    """
cursor.execute(q)
cursor.fetchmany(2)

[(0,
  'german',
  'German',
  '8uBE9ZgfZOM2z_Rllw478Q',
  'Stadtklause',
  '€',
  172,
  4.5,
  '+493051056381',
  2205.244050414826,
  52.504779,
  13.380618,
  'Berlin',
  '10963',
  'Bernburger Str. 35'),
 (1,
  'modern_european',
  'Modern European',
  '6BCLgz8j6j2em7XLLsXoow',
  'Lorenz Adlon Esszimmer',
  '€€€€',
  67,
  5.0,
  '+493022611960',
  969.3816096240975,
  52.516109,
  13.380014,
  'Berlin',
  '10117',
  'Unter den Linden 77')]

<IPython.core.display.Javascript object>

✔️ Load the data

We have successfully loaded the raw data into a SQL database. The next step will be to normalize the database into its final form. Using a database manager we can commit different SQL commands and manipulate the tables easier than using python code from a jupyter notebook

<p align='center'>
  <img alt="database" src="./images/database.png">
</p>